In [ ]:
import os
import sys

import cv2
import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../")
from src.adnet import compute_landmarks

%load_ext autoreload
%autoreload 2

In [ ]:
img_p = "../data/c-07-twofaces_cropped.png"
model_p = "../checkpoints/adnet/adnet_ofiq.onnx"
landmarks = compute_landmarks(img_p, model_p)
# print(landmarks.shape)

img = cv2.imread(img_p)
for idx in range(len(landmarks)):
    x, y = landmarks[idx]
    cv2.circle(img, (x, y), 3, (255, 0, 0), cv2.FILLED)
    cv2.putText(img, str(idx), (x + 2, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
def draw_points(points: np.ndarray, name: str = "default"):
    for idx in range(len(points)):
        x, y = points[idx]
        cv2.circle(img, (x, y), 3, (255, 0, 0), cv2.FILLED)
        cv2.putText(img, name, (x + 2, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

In [ ]:
def get_face_mask(landmarks: np.ndarray, height: int, width: int, alpha: float):

    left_eye_corners = np.stack([landmarks[60], landmarks[64]])
    right_eye_corners = np.stack([landmarks[68], landmarks[72]])

    eye_corners = np.concatenate([left_eye_corners, right_eye_corners])
    eyes_midpoint = np.sum(eye_corners, axis=0) / len(eye_corners)
    eyes_midpoint = eyes_midpoint.astype("int")
    eyes_midpoint = eyes_midpoint[None, :]  # Add extra dimension

    chin = landmarks[16]
    chin = chin[None, :]  # Add extra dimension
    contour_indices = [0, 7, 25, 32]
    contour_points = []
    for idx in contour_indices:
        contour_points.append(landmarks[idx])
    contour = np.array(contour_points)

    # draw_points([eyes_midpoint], "eyes_midpoint")
    # draw_points([chin], "chin")
    # draw_points(contour, "contour")
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.show()

    chin_midpoint_vector = eyes_midpoint - chin
    top_of_forehead = eyes_midpoint + alpha * chin_midpoint_vector
    ellipse_points = np.concatenate([contour, chin, top_of_forehead])
    ellipse_points = np.array(ellipse_points, dtype=np.int32)
    fitted_ellipse = cv2.fitEllipse(ellipse_points)
    center = (int(fitted_ellipse[0][0]), int(fitted_ellipse[0][1]))  # Ellipse center (x, y)
    axes = (
        int(fitted_ellipse[1][0] / 2),
        int(fitted_ellipse[1][1] / 2),
    )  # Semi-major and semi-minor axes
    angle = int(fitted_ellipse[2])  # Rotation angle
    poly_points = cv2.ellipse2Poly(center, axes, angle, 0, 360, 10)

    # Discard ellipse points which are not on forehead
    poly_points_list = []
    chin_midpoint_vector = chin_midpoint_vector.squeeze()
    for p in poly_points:
        if np.dot(p - chin, chin_midpoint_vector) > 1.1 * np.dot(
            chin_midpoint_vector, chin_midpoint_vector
        ):
            poly_points_list.append(p)
    poly_points = np.array(poly_points_list, dtype=np.int32)

    # Add poly_points to landmark points
    landmarks = np.concatenate([landmarks, poly_points])
    landmarks = landmarks.astype("int")

    # Fit convex hull
    hull_points = cv2.convexHull(landmarks).squeeze()

    draw_points(hull_points, "hl")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

    # rect = cv2.boundingRect(hull_points)
    # b = int(rect - rect * 0.05)
    # d = int(rect + rect * 1.05)
    # a = int(rect + rect / 2.0 - (d - b) / 2.0)
    # c = int(rect + rect / 2.0 + (d - b) / 2.0)

    # img_size = 224
    # hull_points = [(int((p - a) / (d - b) * img_size), int((p - b) / (d - b) * img_size)) for p in hull_points]

    # mask = np.zeros((img_size, img_size), dtype=np.uint8)
    # cv2.fillConvexPoly(mask, np.array(hull_points, dtype=np.int32), 1)

    # face_region = np.zeros((height, width), dtype=np.uint8)
    # mask_rescaled = cv2.resize(mask, (c - a, d - b), interpolation=cv2.INTER_NEAREST)

    # left, top, right, bottom = 0, 0, mask_rescaled.shape, mask_rescaled.shape
    # an, bn, cn, dn = a, b, c, d

    # if a < 0:
    #     left -= a
    #     an = 0
    # if c > width:
    #     right -= c - width
    #     cn = width
    # if b < 0:
    #     top -= b
    #     bn = 0
    # if d > height:
    #     bottom -= d - height
    #     dn = height

    # crop = mask_rescaled[top:bottom, left:right]
    # face_region[bn:dn, an:cn] = crop

    # return face_region


img = cv2.imread(img_p)
width, height, channels = img.shape
get_face_mask(landmarks, height, width, alpha=0.0)